## This notebook performs the following steps for inference:
* Read pixel data from dicom images 
* Preprocess images with normalization and standardization + make image 224, 224, 3 to suit the model input format
* Load our built and trained model
* Predict whether the patient has pneumonia from his or her chest X-ray

In [1]:
# Import dependencies
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from skimage.transform import rescale 

Using TensorFlow backend.


In [2]:
def check_dicom(filename): 
    
    """Reads in a .dcm file and returns a numpy array of just the imaging data"""
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    
    ## TO DO:
    ## Check other field of the dicom to make sure the file agrees with our expected input
    
    return img
    
def preprocess_image(img,img_mean,img_std,img_size):
    
    """Takes the numpy array output by check_dicom and runs the appropriate pre-processing needed for our model input"""
    
    img = img / 255.0
    gray_img = (img - img_mean) / img_std
    gray_img = rescale(gray_img, 0.21875, anti_aliasing=True)
    
    proc_img = np.zeros((224, 224, 3))
    
    proc_img[:, :, 0] = gray_img
    proc_img[:, :, 1] = gray_img
    proc_img[:, :, 2] = gray_img
    proc_img = np.resize(proc_img, (1, 224, 224, 3))
    
    return proc_img
  
def load_model(model_path, weight_path):
    
    """Loads in our trained model w/ weights and compiles it"""
    
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    
    my_model = keras.models.model_from_json(loaded_model_json)
    
    my_model.load_weights(weight_path)
    
    return my_model

# This function uses our device's threshold parameters to 
def predict_image(model, img, thresh): 
    
    """Predict whether or not the image shows the presence of pneumonia using our trained model"""
    
    result = model.predict(img)
    predict = result[0]
    prediction = 'No Pneumonia'
    
    if (predict > thresh):
        prediction = 'Pneumonia'
    
    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = 'xray_class_my_model.best.hdf5'

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0.53
img_std = 0.24

my_model = load_model(model_path, weight_path)
thresh = 0.38

## Use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    
    print(pred)

Load file test1.dcm ...
No Pneumonia
Load file test2.dcm ...
No Pneumonia
Load file test3.dcm ...
No Pneumonia
Load file test4.dcm ...
No Pneumonia
Load file test5.dcm ...
No Pneumonia
Load file test6.dcm ...
No Pneumonia
